In [32]:
import pandas as pd
import numpy as np

In [56]:
data = pd.read_csv('survey.csv')
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-null   object
 13  care_options               1259 non-null   object
 14  wellness

In [57]:
df.head(1)

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN


In [58]:
missing_counts = df.isnull().sum()
print(missing_counts)

Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_employees                    0
remote_work                     0
tech_company                    0
benefits                        0
care_options                    0
wellness_program                0
seek_help                       0
anonymity                       0
leave                           0
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical              0
obs_consequence                 0
comments                     1095
dtype: int64


In [59]:
df = df.drop(['comments'], axis=1)
df = df.drop(['work_interfere'], axis=1)
df = df.drop(['state'], axis=1)

In [60]:
print(df['self_employed'].value_counts())

No     1095
Yes     146
Name: self_employed, dtype: int64


In [61]:
mode = df['self_employed'].mode().values[0]
df['self_employed'].fillna(mode, inplace = True)

In [62]:
print(df.isnull().sum())

Timestamp                    0
Age                          0
Gender                       0
Country                      0
self_employed                0
family_history               0
treatment                    0
no_employees                 0
remote_work                  0
tech_company                 0
benefits                     0
care_options                 0
wellness_program             0
seek_help                    0
anonymity                    0
leave                        0
mental_health_consequence    0
phys_health_consequence      0
coworkers                    0
supervisor                   0
mental_health_interview      0
phys_health_interview        0
mental_vs_physical           0
obs_consequence              0
dtype: int64


In [63]:
for row in list(df.columns.values):
    print(row)
    print(df[row].unique())
    print()

Timestamp
['2014-08-27 11:29:31' '2014-08-27 11:29:37' '2014-08-27 11:29:44' ...
 '2015-11-07 12:36:58' '2015-11-30 21:25:06' '2016-02-01 23:04:31']

Age
[         37          44          32          31          33          35
          39          42          23          29          36          27
          46          41          34          30          40          38
          50          24          18          28          26          22
          19          25          45          21         -29          43
          56          60          54         329          55 99999999999
          48          20          57          58          47          62
          51          65          49       -1726           5          53
          61           8          11          -1          72]

Gender
['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'F

In [64]:
# Age replacement
replace_value = df['Age'].mode()

df.loc[df['Age'] > 100, 'Age'] = replace_value
df.loc[df['Age'] < 0, 'Age'] = replace_value

In [65]:
print(df['Gender'].value_counts())

Male                                              615
male                                              206
Female                                            121
M                                                 116
female                                             62
F                                                  38
m                                                  34
f                                                  15
Make                                                4
Male                                                3
Woman                                               3
Cis Male                                            2
Man                                                 2
Female (trans)                                      2
Female                                              2
Trans woman                                         1
msle                                                1
male leaning androgynous                            1
Neuter                      

In [66]:
def replace_male(value):
  if value in ['male', 'M', 'm', 'Make', 'Man']:
    return 'Male'
  else:
    return value

def replace_female(value):
  if value in ['female', 'F', 'f', 'Woman', 'Man']:
    return 'Female'
  else:
    return value

df['Gender'] = df['Gender'].map(replace_male)
df['Gender'] = df['Gender'].map(replace_female)

In [67]:
print(df['Gender'].value_counts())

Male                                              977
Female                                            239
Male                                                3
Female (trans)                                      2
Female                                              2
Cis Male                                            2
Cis Man                                             1
Guy (-ish) ^_^                                      1
male leaning androgynous                            1
Trans woman                                         1
msle                                                1
Neuter                                              1
queer                                               1
femail                                              1
cis-female/femme                                    1
Mail                                                1
cis male                                            1
A little about you                                  1
Malr                        

In [72]:
df.loc[~df['Gender'].isin(['Male', 'Female']), 'Gender'] = np.nan

<b>What is the distribution of mental health conditions among different age groups in the tech industry?

In [77]:
bins = [0, 20, 40, 60, 80] 
labels = ['0-20', '21-40', '41-60', '60+']
df['Age_group'] = pd.cut(df['Age'], bins=bins, labels=labels)

In [133]:
columns = ['Age_group', 'mental_health_consequence']#, 'phys_health_consequence', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical']
df_tech = df.loc[df['tech_company'] == 'Yes'] 
df_tech = df_tech[columns]
#print(df_tech.head())

for col in columns:
    if col != 'Age_group':
        print(col)
        print(df_tech.groupby('Age_group')[col].value_counts(normalize=True) * 100)

mental_health_consequence
Age_group  mental_health_consequence
0-20       No                           59.090909
           Yes                          27.272727
           Maybe                        13.636364
21-40      No                           42.105263
           Maybe                        36.954087
           Yes                          20.940649
41-60      Maybe                        36.936937
           No                           36.936937
           Yes                          26.126126
60+        Maybe                        50.000000
           Yes                          50.000000
Name: mental_health_consequence, dtype: float64


<b>How does the frequency of mental health issues vary by gender?

In [132]:
columns = ['Gender', 'mental_health_consequence']#, 'phys_health_consequence', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical']
df_gender = df[columns]

for col in columns:
    if col != 'Gender':
        print(col)
        print(df_gender.groupby('Gender')[col].value_counts(normalize=True) * 100)
        

mental_health_consequence
Gender  mental_health_consequence
Female  Maybe                        43.514644
        No                           29.707113
        Yes                          26.778243
Male    No                           41.965200
        Maybe                        36.438076
        Yes                          21.596725
Name: mental_health_consequence, dtype: float64


<b>Identify the countries with the highest and lowest reported rates of mental health issues in the tech industry.

In [131]:
columns = ['Country', 'mental_health_consequence']#, 'phys_health_consequence', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical']
df_country = df.loc[df['tech_company'] == 'Yes'] 
df_country = df[columns]

df_country_aux = pd.DataFrame(df_country.loc[df_country['mental_health_consequence'] == 'Yes'].groupby('Country')[col].agg('count'))

max_row = df_country_aux['mental_health_consequence'].idxmax()
print(max_row)

min_row = df_country_aux['mental_health_consequence'].idxmin()
print(min_row)

United States
Bahamas, The
